## Läs in filen, skriv ut, och städa den

In [1]:
import pandas as pd

import os

data = pd.read_excel("Webförs.xlsx")

os.startfile("Webförs.xlsx", "print")

datum = input("Vilket datum?")

data.dropna(subset=['ProductName'], inplace = True)

data["OrderNo"] = data["OrderNo"].astype(int).replace('\.o', '', regex = True)

data["PriceIncVat"] = data["PriceIncVat"].astype(int).replace('\.o', '', regex = True)

data = data[["ProductName", "OrderNo", "NoOfItems", "PriceIncVat", "PriceExcVat", "Info"]]


Vilket datum?222


## Kontering Klarna och kort

In [2]:
data2 = data.copy()

data2["Belopp"] = None

data2["Konto"] = "1570"

data2["B"] = 0
data2["C"]= None
data2["D"] = None
data2["E"] = None

data2.loc[data2['Info'].str.contains('Kort'), 'Konto'] = "1571"

data2.loc[data2['Konto'].str.contains('1571'), 'OrderNo'] = datum + " Webfors"

## Kontering Frakt

In [3]:
t = data2[data2.Info.str.contains('Hemleverans')].drop_duplicates(subset='OrderNo')

t2 = data2[data2.Info.str.contains('Manuell')].drop_duplicates(subset='OrderNo')

t2 = t2['PriceIncVat'].apply(lambda x: x+250)

t = t['PriceIncVat'].apply(lambda x: x+19)

def frakt(data):
    try:
        antal = t.count()
        split = [
        {'Konto':3507, 'B': 920, 'Belopp': -15.2*antal, 'Referens': datum + ' Webfors'},
        {'Konto':2610, 'B': 0, 'Belopp': -3.8*antal, 'Referens': 'Webfors'},
        ] 
        split = pd.DataFrame(split)
        split["C"]= None
        split["D"] = None
        split["E"] = None
    except:
        pass
    return split[["Konto", "B", "C", "D", "E", "Belopp", "Referens"]]

def frakt2(data):
    try:
        antal = t2.count()
        split = [
        {'Konto':3507, 'B': 920, 'Belopp': -200*antal, 'Referens': datum + ' Webfors'},
        {'Konto':2610, 'B': 0, 'Belopp': -50*antal, 'Referens': 'Webfors'},
        ] 
        split = pd.DataFrame(split)
        split["C"]= None
        split["D"] = None
        split["E"] = None
    except:
        pass
    return split[["Konto", "B", "C", "D", "E", "Belopp", "Referens"]]

data3 = frakt(data)

data4 = frakt2(data)

my_list= t.index.values.tolist()

my_list2= t2.index.values.tolist()

data2.at[my_list,"PriceIncVat"]= t

data2.at[my_list2,"PriceIncVat"]= t2

## Strukturera data2 (Klarna, Kort + frakt)

In [4]:
data2 = data2[["Konto", "B", "C", "D", "E", "PriceIncVat", "OrderNo"]]

data2.columns = ["Konto", "B", "C", "D", "E","Belopp", "Referens"]

## Kontering produkter

In [5]:
data["Moms"] = data["PriceIncVat"]-data["PriceExcVat"]

data["Konto"] = None

data["Referens"] = None

data["Konto"] = "2976"

def gruppaket_split(data):

    try:
        antal = data.NoOfItems[data.ProductName == 'Stora Gruppaketet'].sum()
        split = [
        {'Konto':2976, 'Belopp': -380*antal, 'Referens': '3054/110'},
        {'Konto':2976, 'Belopp': 96*antal, 'Referens': '3069/110'},
        {'Konto':2979, 'Belopp': -80*antal, 'Referens': '3923/110'},
        {'Konto':2610, 'Belopp': -71*antal, 'Referens': 'Webfors'},
        ] 
        split = pd.DataFrame(split)
        split["B"] = 0
        split["C"]= None
        split["D"] = None
        split["E"] = None
    except:
        pass
    
    return split[["Konto", "B", "C", "D", "E", "Belopp", "Referens"]]

def guldkort_ÅA_split(data):
    try:
        antal = data.NoOfItems[data.ProductName == 'Guldkortet ÅA'].sum()
        split = [
        {'Konto': 2976, 'Belopp': -1359.2*antal, 'Referens': '3056/110'},
        {'Konto': 2976, 'Belopp': 80*antal, 'Referens': '3069/110'},
        {'Konto': 2610, 'Belopp': -319.8*antal, 'Referens': 'Webfors'}
        ]
        split = pd.DataFrame(split)
        split["B"] = 0
        split["C"]= None
        split["D"] = None
        split["E"] = None
    except:
        pass
    
    return split[["Konto", "B", "C", "D", "E", "Belopp", "Referens"]]

data5 = gruppaket_split(data)

data6 = guldkort_ÅA_split(data)

data.loc[data['ProductName'].str.contains('Guldkortet'), 'Referens'] = '3056/110'
data.loc[data['ProductName'].str.contains('Gröna Kortet'), 'Referens'] = '3018/110'
data.loc[data['ProductName'].str.contains('Gröna kortet'), 'Referens'] = '3018/110'
data.loc[data['ProductName'].str.contains('Åkband'), 'Referens'] = '3051/110'
data.loc[data['ProductName'].str.contains('Paketentré'), 'Referens'] = '3001/110'
data.loc[data['ProductName'].str.contains('Entré'), 'Referens'] = '3001/110'
data.loc[data['ProductName'].str.contains('House'), 'Referens'] = '3059/110'
data.loc[data['ProductName'].str.contains('Rab'), 'Referens'] = '3011/110'
data.loc[data['ProductName'].str.contains('Stora'), 'Referens'] = 'Stora Gruppaketet'
data.loc[data['ProductName'].str.contains('Guldkortet ÅA'), 'Referens'] = 'ÅA'
data.loc[data['ProductName'].str.contains('Present'), 'Referens'] = datum + " Webfors"
data.loc[data['Info'].str.contains('Hämta'), 'Referens'] = datum + " Webfors"

data.loc[data['ProductName'].str.contains('House'), 'Konto'] = "2978"
data.loc[data['ProductName'].str.contains('Present'), 'Konto'] = "2894"
data.loc[data['ProductName'].str.contains('Stora'), 'Konto'] = "1915"
data.loc[data['ProductName'].str.contains('Guldkortet ÅA'), 'Konto'] = "1915"
data.loc[data['Info'].str.contains('Hämta'), 'Konto'] = "2813"

data["2610"] = None
data["2630"] = None

data["Belopp"] = None

data.loc[data['Konto'].str.contains('2976'), '2610'] = data["Moms"]*-1
data.loc[data['Konto'].str.contains('2978'), '2630'] = data["Moms"]*-1

data.loc[data['Konto'].str.contains('2976'), 'Belopp'] = data["PriceExcVat"]*-1
data.loc[data['Konto'].str.contains('2978'), 'Belopp'] = data["PriceExcVat"]*-1
data.loc[data['Konto'].str.contains('2813'), 'Belopp'] = data["PriceIncVat"]*-1
data.loc[data['Konto'].str.contains('1915'), 'Belopp'] = data["PriceIncVat"]*-1
data.loc[data['Konto'].str.contains('2894'), 'Belopp'] = data["PriceIncVat"]*-1

Moms_25 = data["2610"].sum()
Moms_6 = data["2630"].sum()

data["B"] = 0
data["C"]= None
data["D"] = None
data["E"] = None

data = data[["Konto", "B", "C", "D", "E", "Belopp", "Referens"]]

data.loc[-1] = ["2610",0, None, None, None, Moms_25, 'Webfors']
data.loc[-2] = ["2630",0, None, None, None, Moms_6, 'Webfors']


## Kontering presentkort microdeb

In [6]:
pk = pd.read_excel("Presentkort.xlsx")

pk = pk[pk['Operatör'].str.contains("Jetshop")]

pk['Konto'] = "1570"
pk["B"] = 0
pk["C"]= None
pk["D"] = None
pk["E"] = None

pk = pk[["Konto", "B", "C", "D", "E", "Bruttobelopp", "Kvittonr."]]

pk.columns = ["Konto", "B", "C", "D", "E","Belopp", "Referens"]

summa_pk = pk['Belopp'].sum()*-1

pk.loc[-1] = ["2894", 0, None, None, None, summa_pk, datum + " Microdeb" ]

## Sammanställning dataframes, städning, samt skapa csv-fil

In [7]:
frames = [data, data2, data3, data4, data5, data6, pk]

data7 = pd.concat(frames)

data7 = data7[data7.Referens != 'Stora Gruppaketet']

data7 = data7[data7.Referens != 'ÅA']

data7 = data7[data7.Belopp != 0]

data7["Belopp"] = data7["Belopp"].astype(float).replace('\.o', '', regex = True)

data7["Konto"] = data7["Konto"].astype(int).replace('\.o', '', regex = True)

data7.B = data7.B.astype(int) 

data7['Referens'] = data7['Referens'].apply(lambda x: 'Webfors' if x is None else x)

data8 = data7.groupby(['Referens', 'Konto', 'B'])['Belopp'].sum().reset_index()

data8["C"]= None
data8["D"] = None
data8["E"] = None

data8 = data8[["Konto", 'B', 'C', 'D', 'E','Belopp', "Referens"]]

data8.to_csv('Webförs.csv', index=False, header=False, sep = ";") 